In [11]:
!pip install pillow


   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ------------- -------------------------- 2.4/7.0 MB 14.9 MB/s eta 0:00:01
   ---------------------------------------- 7.0/7.0 MB 20.5 MB/s eta 0:00:00


In [5]:
import os
import shutil
import pandas as pd
from tqdm import tqdm

# Paths
metadata_file = 'HAM10000_metadata.csv'
images_dir = 'all_images'         # Folder with all extracted .jpg files
output_base = 'data'              # Where to make train/val folders

# Create train/val folders and class subfolders
classes = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']
for split in ['train', 'val']:
    for cls in classes:
        os.makedirs(os.path.join(output_base, split, cls), exist_ok=True)
    



In [ ]:
# Read metadata
df = pd.read_csv(metadata_file)

# Stratified split
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['dx'], random_state=42)

    # Function to copy images into subfolders
def copy_images(sub_df, split):
    for _, row in tqdm(sub_df.iterrows(), total=len(sub_df), desc=f'Processing {split}'):
        fname = row['image_id'] + '.jpg'
        label = row['dx']
        src = os.path.join(images_dir, fname)
        dst = os.path.join(output_base, split, label, fname)
        if os.path.exists(src):
            shutil.copy(src, dst)
# Run the automation
copy_images(train_df, 'train')
copy_images(val_df, 'val')


Processing val: 100%|██████████| 2003/2003 [00:24<00:00, 80.29it/s] 


In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import PIL
# Load training data
train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True
).flow_from_directory('data/train', target_size=(224,224), batch_size=32, class_mode='categorical')

# Load validation data
val_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    'data/val', target_size=(224,224), batch_size=32, class_mode='categorical')

# Base model
base = tf.keras.applications.MobileNetV2(input_shape=(224,224,3), include_top=False, weights='imagenet')
base.trainable = False

# Add head
x = tf.keras.layers.GlobalAveragePooling2D()(base.output)
x = tf.keras.layers.Dropout(0.3)(x)
output = tf.keras.layers.Dense(7, activation='softmax')(x)

model = tf.keras.Model(inputs=base.input, outputs=output)

# Compile
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train
model.fit(train_gen, validation_data=val_gen, epochs=10)

# Save
model.save('skin_model.h5')
print("✅ Model trained and saved to skin_model.h5")


Found 8012 images belonging to 7 classes.
Found 2003 images belonging to 7 classes.
Epoch 1/10
251/251 [==============================] - 202s 771ms/step - loss: 1.0433 - accuracy: 0.6665 - val_loss: 0.8037 - val_accuracy: 0.7089
Epoch 2/10
251/251 [==============================] - 117s 467ms/step - loss: 0.8511 - accuracy: 0.7096 - val_loss: 0.7616 - val_accuracy: 0.7189
Epoch 3/10
251/251 [==============================] - 115s 459ms/step - loss: 0.8011 - accuracy: 0.7224 - val_loss: 0.7580 - val_accuracy: 0.7299
Epoch 4/10
251/251 [==============================] - 116s 462ms/step - loss: 0.7776 - accuracy: 0.7223 - val_loss: 0.7448 - val_accuracy: 0.7324
Epoch 5/10
251/251 [==============================] - 116s 460ms/step - loss: 0.7600 - accuracy: 0.7279 - val_loss: 0.7265 - val_accuracy: 0.7439
Epoch 6/10
251/251 [==============================] - 120s 476ms/step - loss: 0.7409 - accuracy: 0.7353 - val_loss: 0.7340 - val_accuracy: 0.7369
Epoch 7/10
251/251 [====================

In [2]:
base.trainable = True
for layer in base.layers[:-50]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(train_gen, validation_data=val_gen, epochs=5)


Epoch 1/5
251/251 [==============================] - 127s 492ms/step - loss: 1.0604 - accuracy: 0.6618 - val_loss: 0.8394 - val_accuracy: 0.7399
Epoch 2/5
251/251 [==============================] - 125s 499ms/step - loss: 0.8906 - accuracy: 0.7008 - val_loss: 0.8786 - val_accuracy: 0.7439
Epoch 3/5
251/251 [==============================] - 127s 504ms/step - loss: 0.8304 - accuracy: 0.7157 - val_loss: 0.8536 - val_accuracy: 0.7474
Epoch 4/5
251/251 [==============================] - 127s 505ms/step - loss: 0.7594 - accuracy: 0.7346 - val_loss: 0.8382 - val_accuracy: 0.7524
Epoch 5/5
251/251 [==============================] - 129s 512ms/step - loss: 0.7326 - accuracy: 0.7395 - val_loss: 0.7970 - val_accuracy: 0.7604


In [3]:
model.save('skin_model.h5')

In [ ]:
exit()

: 